In [2]:
import numpy as np
import anndata
import scanpy as sc
import umap
import umap.plot
import igraph as ig
import leidenalg
import matplotlib.pyplot as plt
import scanpy.external as sce

/home/alopez/anaconda3/envs/r_env/lib/python3.11/site-packages/numba/np/ufunc/dufunc.py:343: NumbaWarning: Compilation requested for previously compiled argument types ((uint32,)). This has no effect and perhaps indicates a bug in the calling code (compiling a ufunc more than once for the same signature
  warnings.warn(msg, errors.NumbaWarning)
/home/alopez/anaconda3/envs/r_env/lib/python3.11/site-packages/numba/np/ufunc/dufunc.py:343: NumbaWarning: Compilation requested for previously compiled argument types ((uint32,)). This has no effect and perhaps indicates a bug in the calling code (compiling a ufunc more than once for the same signature
  warnings.warn(msg, errors.NumbaWarning)
/home/alopez/anaconda3/envs/r_env/lib/python3.11/site-packages/numba/np/ufunc/dufunc.py:343: NumbaWarning: Compilation requested for previously compiled argument types ((uint32,)). This has no effect and perhaps indicates a bug in the calling code (compiling a ufunc more than once for the same signature
 

In [3]:
# Withou Feature Selection
# Filtered data
filtered_data = anndata.read_h5ad("./data/filtered_data.h5ad.gz")
print(f"Data loaded (filter): {filtered_data.shape} cells x genes")
# Filtered and normalized data
normalized_data = anndata.read_h5ad("./data/filtered_normalized_data.h5ad.gz")
print(f"Data loaded (filter + normalized ): {normalized_data.shape} cells x genes")
# Filtered + normalized + scaled data
scaled_data = anndata.read_h5ad("./data/scaled_data.h5ad.gz")
print(f"Data loaded (filter + normalized + scaled ): {scaled_data.shape} cells x genes")

Data loaded (filter): (10101, 32738) cells x genes
Data loaded (filter + normalized ): (10101, 32738) cells x genes
Data loaded (filter + normalized + scaled ): (10101, 32738) cells x genes


In [2]:
# Load AnnData objects
HVG_data = anndata.read_h5ad("./data/HVG_data.h5ad.gz")
print(f"Data loaded (Filtered + HVG): {HVG_data.shape} cells x genes")
pcHVG_data = anndata.read_h5ad("./data/pcHVG_data.h5ad.gz")
print(f"Data loaded (Filtered + HVG + ZScore): {pcHVG_data.shape} cells x genes")

Data loaded (Filtered + HVG): (10101, 919) cells x genes
Data loaded (Filtered + HVG + ZScore): (10101, 919) cells x genes


# 1. Batch effect

Problem: Variation in single-cell and spatial RNA sequencing data is known to be influenced by technical factors. In some cases, these technical factors may confound our ability to measure true biological variation between samples, making it more challenging to address the research question at hand.

Cause: These confounding factors include experimental biases and batch effects. Unavoidable systematic technical biases can include unequal amplification during PCR, cell lysis, reverse transcriptase enzyme efficiency, and stochastic molecular sampling during sequencing. By contrast, batch effects are technical, non-biological factors that also affect variation in the resulting data, but they occur in batches of samples. A “batch” refers to an individual group of samples that are processed differently relative to other samples in the experiment.

Solution: Technical factors that potentially lead to batch effects may be avoided with mitigation strategies in the lab and during sequencing. Examples of lab strategies include: sampling cells on the same day, using the same handling personnel, reagent lots, protocols, reducing PCR amplification bias, and generally using the same equipment. Sequencing strategies can include multiplexing libraries across flow cells. For example, if samples came from two patients, pooling libraries together and spreading them across flow cells can potentially spread out the flow cell-specific variation across samples.

Computational batch correction aims to remove technical variation from the data preventing this variation from confounding downstream analysis. There are several batch correction methods and tools that have implemented them.

Methods: 

 * https://www.10xgenomics.com/analysis-guides/introduction-batch-effect-correction
 * Harmony: https://doi.org/10.1038%2Fs41592-019-0619-0
 * BBKNN: https://doi.org/10.1093/bioinformatics/btz625
 * Scanorama: 
 * LIGER

Large single-cell RNA sequencing (scRNA-seq) projects usually need to generate data across multiple batches due to logistical constraints. However, the processing of different batches is often subject to uncontrollable differences, e.g., changes in operator, differences in reagent quality. This results in systematic differences in the observed expression in cells from different batches, which we refer to as “batch effects”. Batch effects are problematic as they can be major drivers of heterogeneity in the data, masking the relevant biological differences and complicating interpretation of the results.


Normalization occurs regardless of the batch structure and only considers technical biases, while batch correction - as the name suggests - only occurs across batches and must consider both technical biases and biological differences. Technical biases tend to affect genes in a similar manner, or at least in a manner related to their biophysical properties (e.g., length, GC content), while biological differences between batches can be highly unpredictable

In [3]:
## Harmony, BBKNN and Scanorama need PCA
# Without Z-Score
sc.tl.pca(HVG_data, n_comps=50, svd_solver='randomized')
# With Z-Score
sc.tl.pca(pcHVG_data, n_comps=50, svd_solver='randomized')

In [4]:
sc.tl.pca(filtered_data, n_comps=50, svd_solver='randomized')
sc.tl.pca(normalized_data, n_comps=50, svd_solver='randomized')
sc.tl.pca(scaled_data, n_comps=50, svd_solver='randomized')

## 1.1 Harmony

In [6]:
# Without ZScore ---------------------------------------------------------------------------------------------------

# Reprogramming type is the key to differentiate between experiments
HVG_data_harmony = HVG_data.copy()
sce.pp.harmony_integrate(HVG_data_harmony,"reprogramming_type")
# Save Harmony PCA in X_pca
HVG_data_harmony.obsm['X_pca'] = HVG_data_harmony.obsm['X_pca_harmony']

HVG_data_harmony.write_h5ad("./data/batch_effect/Harmony_HVG_data.h5ad.gz", compression="gzip")

2024-07-04 17:32:53,673 - harmonypy - INFO - Computing initial centroids with sklearn.KMeans...


2024-07-04 17:32:55,866 - harmonypy - INFO - sklearn.KMeans initialization complete.
2024-07-04 17:32:55,922 - harmonypy - INFO - Iteration 1 of 10
2024-07-04 17:32:57,800 - harmonypy - INFO - Iteration 2 of 10
2024-07-04 17:32:59,755 - harmonypy - INFO - Converged after 2 iterations


In [7]:
# With ZScore ---------------------------------------------------------------------------------------------------

# Reprogramming type is the key to differentiate between experiments
pcHVG_data_harmony = pcHVG_data.copy()
sce.pp.harmony_integrate(pcHVG_data_harmony,"reprogramming_type")
pcHVG_data_harmony.obsm['X_pca'] = pcHVG_data_harmony.obsm['X_pca_harmony']

pcHVG_data_harmony.write_h5ad("./data/batch_effect/Harmony_pcHVG_data.h5ad.gz", compression="gzip")

2024-07-04 17:33:01,674 - harmonypy - INFO - Computing initial centroids with sklearn.KMeans...


2024-07-04 17:33:03,673 - harmonypy - INFO - sklearn.KMeans initialization complete.
2024-07-04 17:33:03,736 - harmonypy - INFO - Iteration 1 of 10
2024-07-04 17:33:05,673 - harmonypy - INFO - Iteration 2 of 10
2024-07-04 17:33:07,503 - harmonypy - INFO - Converged after 2 iterations


In [5]:
# Without Feature Selection ---------------------------------------------------------------------------------------------------

# Reprogramming type is the key to differentiate between experiments
filtered_data_harmony = filtered_data.copy()
sce.pp.harmony_integrate(filtered_data_harmony,"reprogramming_type")
filtered_data_harmony.obsm['X_pca'] = filtered_data_harmony.obsm['X_pca_harmony']
filtered_data_harmony.write_h5ad("./data/batch_effect/Harmony_filtered_data.h5ad.gz", compression="gzip")

# Reprogramming type is the key to differentiate between experiments
normalized_data_harmony = normalized_data.copy()
sce.pp.harmony_integrate(normalized_data_harmony,"reprogramming_type")
normalized_data_harmony.obsm['X_pca'] = normalized_data_harmony.obsm['X_pca_harmony']
normalized_data_harmony.write_h5ad("./data/batch_effect/Harmony_normalized_data.h5ad.gz", compression="gzip")

# Reprogramming type is the key to differentiate between experiments
scaled_data_harmony = scaled_data.copy()
sce.pp.harmony_integrate(scaled_data_harmony,"reprogramming_type")
scaled_data_harmony.obsm['X_pca'] = scaled_data_harmony.obsm['X_pca_harmony']
scaled_data_harmony.write_h5ad("./data/batch_effect/Harmony_scaled_data.h5ad.gz", compression="gzip")

2024-07-11 16:55:42,507 - harmonypy - INFO - Computing initial centroids with sklearn.KMeans...
2024-07-11 16:55:44,600 - harmonypy - INFO - sklearn.KMeans initialization complete.
2024-07-11 16:55:44,658 - harmonypy - INFO - Iteration 1 of 10
2024-07-11 16:55:46,484 - harmonypy - INFO - Iteration 2 of 10
2024-07-11 16:55:48,308 - harmonypy - INFO - Iteration 3 of 10
2024-07-11 16:55:50,133 - harmonypy - INFO - Converged after 3 iterations
2024-07-11 16:56:00,451 - harmonypy - INFO - Computing initial centroids with sklearn.KMeans...
2024-07-11 16:56:02,628 - harmonypy - INFO - sklearn.KMeans initialization complete.
2024-07-11 16:56:02,683 - harmonypy - INFO - Iteration 1 of 10
2024-07-11 16:56:04,595 - harmonypy - INFO - Iteration 2 of 10
2024-07-11 16:56:06,518 - harmonypy - INFO - Converged after 2 iterations
2024-07-11 16:56:16,628 - harmonypy - INFO - Computing initial centroids with sklearn.KMeans...
2024-07-11 16:56:18,698 - harmonypy - INFO - sklearn.KMeans initialization comp

## 1.2 BBKNN

In [8]:
# Without Z-Score
HVG_data_BBKNN = HVG_data.copy()
sc.external.pp.bbknn(HVG_data_BBKNN, batch_key="reprogramming_type")  # running bbknn 1.3.6
HVG_data_BBKNN.write_h5ad("./data/batch_effect/BBKNN_HVG.h5ad_data.gz", compression="gzip")

In [9]:
# With Z-Score
pcHVG_data_BBKNN = pcHVG_data.copy()
sc.external.pp.bbknn(pcHVG_data_BBKNN, batch_key="reprogramming_type")  # running bbknn 1.3.6
pcHVG_data_BBKNN.write_h5ad("./data/batch_effect/BBKNN_pcHVG_data.h5ad.gz", compression="gzip")

In [8]:
# Without Feature Selection ---------------------------------------------------------------------------------------------------

# Reprogramming type is the key to differentiate between experiments
filtered_data_BBKNN = filtered_data.copy()
sc.external.pp.bbknn(filtered_data_BBKNN,"reprogramming_type")
filtered_data_BBKNN.write_h5ad("./data/batch_effect/BBKNN_filtered_data.h5ad.gz", compression="gzip")

# Reprogramming type is the key to differentiate between experiments
normalized_data_BBKNN = normalized_data.copy()
sc.external.pp.bbknn(normalized_data_BBKNN,"reprogramming_type")
normalized_data_BBKNN.write_h5ad("./data/batch_effect/BBKNN_normalized_data.h5ad.gz", compression="gzip")

# Reprogramming type is the key to differentiate between experiments
scaled_data_BBKNN = scaled_data.copy()
sc.external.pp.bbknn(scaled_data_BBKNN,"reprogramming_type")
scaled_data_BBKNN.write_h5ad("./data/batch_effect/BBKNN_scaled_data.h5ad.gz", compression="gzip")

/tmp/ipykernel_1145040/1143968102.py:5: FutureWarning: The specified parameters ('batch_key',) are no longer positional. Please specify them like `batch_key='reprogramming_type'`
  sc.external.pp.bbknn(filtered_data_BBKNN,"reprogramming_type")


/tmp/ipykernel_1145040/1143968102.py:10: FutureWarning: The specified parameters ('batch_key',) are no longer positional. Please specify them like `batch_key='reprogramming_type'`
  sc.external.pp.bbknn(normalized_data_BBKNN,"reprogramming_type")


/tmp/ipykernel_1145040/1143968102.py:15: FutureWarning: The specified parameters ('batch_key',) are no longer positional. Please specify them like `batch_key='reprogramming_type'`
  sc.external.pp.bbknn(scaled_data_BBKNN,"reprogramming_type")


## 1.3 Scanorama

https://www.nature.com/articles/s41587-019-0113-3

In [10]:
# Without Z-Score 
HVG_data_Scanorama = HVG_data.copy()
sce.pp.scanorama_integrate(HVG_data_Scanorama,"reprogramming_type") 
HVG_data_Scanorama.write_h5ad("./data/batch_effect/Scanorama_HVG_data.h5ad.gz", compression="gzip")

[[0.         0.15580789]
 [0.         0.        ]]
Processing datasets False <=> chemical reprogramming


In [11]:
# With Z-Score
pcHVG_data_Scanorama= pcHVG_data.copy()
sce.pp.scanorama_integrate(pcHVG_data_Scanorama,"reprogramming_type") 
pcHVG_data_Scanorama.write_h5ad("./data/batch_effect/Scanorama_pcHVG_data.h5ad.gz", compression="gzip")

[[0.         0.14408042]
 [0.         0.        ]]
Processing datasets False <=> chemical reprogramming


In [9]:
# Without Feature Selection ---------------------------------------------------------------------------------------------------

# Reprogramming type is the key to differentiate between experiments
filtered_data_Scanorama = filtered_data.copy()
sce.pp.scanorama_integrate(filtered_data_Scanorama,"reprogramming_type")
filtered_data_BBKNN.write_h5ad("./data/batch_effect/Scanorama_filtered_data.h5ad.gz", compression="gzip")

# Reprogramming type is the key to differentiate between experiments
normalized_data_Scanorama = normalized_data.copy()
sce.pp.scanorama_integrate(normalized_data_Scanorama,"reprogramming_type")
normalized_data_Scanorama.write_h5ad("./data/batch_effect/Scanorama_normalized_data.h5ad.gz", compression="gzip")

# Reprogramming type is the key to differentiate between experiments
scaled_data_Scanorama = scaled_data.copy()
sce.pp.scanorama_integrate(scaled_data_Scanorama,"reprogramming_type")
scaled_data_Scanorama.write_h5ad("./data/batch_effect/Scanorama_scaled_data.h5ad.gz", compression="gzip")

[[0.         0.11634401]
 [0.         0.        ]]
Processing datasets False <=> chemical reprogramming
[[0.         0.15096798]
 [0.         0.        ]]
Processing datasets False <=> chemical reprogramming
[[0.         0.17572599]
 [0.         0.        ]]
Processing datasets False <=> chemical reprogramming


## 1.4 LIGER

In [ ]:
### CON O SIN PCA ANTES? 
# integrative non-negative matrix factorization on the normalized and scaled datasets
# los datos pueden estar limpios, con los genes seleccionados y normalizado pero no centrados -> HVG

In [4]:
import pyliger

In [ ]:
# Vamos a probar con los datos originales para tener otra aproximación

In [38]:
# Dividimos el objeto de AnnData en dos, uno por cada experimento
HVG_data_1 = HVG_data[HVG_data.obs.cell_state == "somatic", :]
HVG_data_2 = HVG_data[HVG_data.obs.cell_state == "intermediate plastic state with a regeneration-like program", :]

# Check de que en uns tenemos "sample_name" que identifica el experimento -> HVG_data.obs["cell_state"]
HVG_data_1.uns.keys()
HVG_data_1.uns["sample_name"] = "somatic"

HVG_data_2.uns.keys()
HVG_data_2.uns["sample_name"] = "chemical_reprogramming"

# Check de nombre de columnas y filas
print(HVG_data_1.obs.index.name)
HVG_data_1.var.index.name = "gene_name"
print(HVG_data_1.var.index.name)

print(HVG_data_2.obs.index.name)
HVG_data_2.var.index.name = "gene_name"
print(HVG_data_2.var.index.name)

# Check nombre de genes y células
print(HVG_data_1.obs_names)
print(HVG_data_1.var_names)

print(HVG_data_2.obs_names)
print(HVG_data_2.var_names)

# Check de que los identificadores son únicos
print(HVG_data_1.obs_names.duplicated().any())
print(HVG_data_1.var_names.duplicated().any())

print(HVG_data_2.obs_names.duplicated().any())
print(HVG_data_2.var_names.duplicated().any())

cell_id
gene_name
cell_id
gene_name
Index(['AAACCCAAGAAGAGCA-1', 'AAACCCACAAAGTGTA-1', 'AAACCCACACTGTCCT-1',
       'AAACCCAGTCGACTTA-1', 'AAACCCAGTGGTCTTA-1', 'AAACCCATCAGCCTTC-1',
       'AAACGAAAGAGAACCC-1', 'AAACGAAAGCTTACGT-1', 'AAACGAACAATGAAAC-1',
       'AAACGAAGTCCGTTTC-1',
       ...
       'TTTGGTTCATTAAAGG-1', 'TTTGGTTGTCTTTCAT-1', 'TTTGGTTTCAAGCCGC-1',
       'TTTGGTTTCTCTTAAC-1', 'TTTGTTGCATCTCAAG-1', 'TTTGTTGGTATCAGGG-1',
       'TTTGTTGGTATGCAAA-1', 'TTTGTTGTCATTGCGA-1', 'TTTGTTGTCCCGTTGT-1',
       'TTTGTTGTCGACGTCG-1'],
      dtype='object', name='cell_id', length=5372)
Index(['ISG15', 'RNF207', 'CTNNBIP1', 'DHRS3', 'KAZN', 'EFHD2', 'FBLIM1',
       'AKR7A2', 'NBL1', 'CAMK2N1',
       ...
       'PAXBP1', 'ETS2', 'HMGN1', 'COL18A1', 'COL6A2', 'MT-CO1', 'MT-ATP6',
       'MT-CO3', 'MT-ND4L', 'MT-CYB'],
      dtype='object', name='gene_name', length=919)
Index(['AAACCCAAGTCACGAG-1', 'AAACCCATCTAGATCG-1', 'AAACGAAAGCTGACAG-1',
       'AAACGAAAGTCCTACA-1', 'AAACGAACATATCT

/tmp/ipykernel_2458894/1471175330.py:7: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
/tmp/ipykernel_2458894/1471175330.py:10: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.


In [42]:
# Preprocessing and Normalization
adata_list = [HVG_data_1,HVG_data_2]
ifnb_liger = pyliger.create_liger(adata_list)

pyliger.normalize(ifnb_liger)
pyliger.select_genes(ifnb_liger)
pyliger.scale_not_center(ifnb_liger)

# Joint Matrix Factorization
pyliger.optimize_ALS(ifnb_liger, k = 20)

Removing 234 genes not expressing in somatic.


AttributeError: 'Liger' object has no attribute 'var_genes'

In [56]:

from sklearn.decomposition import PCA

# Convertir el objeto AnnData a un DataFrame de pandas
df_data = HVG_data_LIGER.to_df()

# Crear un objeto LIGER
liger = pyLiger.create_liger(df_data)

# Normalizar los datos
# liger.normalize()
# Identificar los genes variables
# liger.select_genes()
# Escalar los datos
# liger.scale()
# Realizar la factorización de matrices no negativa (NMF)
# liger.optimize_nmf(k=20)  # Aquí k es el número de factores latentes

# Integrar los datos usando los factores compartidos
liger.quantile_norm()

# Obtener las representaciones integradas
integrated_data = liger.get_expression_matrix()

# Visualizar los resultados
# Usamos PCA para reducir a 2D para visualización
# pca = PCA(n_components=2)
# pca_result = pca.fit_transform(integrated_data.T)

# Crear etiquetas de lote para visualización
# labels = ['Batch 1'] * data1.shape[0] + ['Batch 2'] * data2.shape[0]

# Convertir a un DataFrame para visualización
# pca_df = pd.DataFrame(pca_result, columns=['PC1', 'PC2'])
# pca_df['Batch'] = labels

# Visualizar los datos integrados
# plt.figure(figsize=(10, 8))
# for label in np.unique(labels):
#     plt.scatter(pca_df[pca_df['Batch'] == label]['PC1'],
#                 pca_df[pca_df['Batch'] == label]['PC2'],
#                 label=label, s=10)
# plt.title('LIGER Integrated Data')
# plt.legend()
# plt.xlabel('PC1')
# plt.ylabel('PC2')
# plt.show()

ModuleNotFoundError: No module named 'pyLiger'

In [4]:
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
from rpy2.robjects.conversion import localconverter

In [6]:
# Instalar paquetes
# install.packages('BiocManager')
# BiocManager::install('ronammar/liger', dependencies=TRUE)

# Convertir el objeto AnnData a un DataFrame de pandas
df_data = HVG_data.to_df()

# Activar la conversión automática entre pandas y R
pandas2ri.activate()

# Importar liger en R
liger = importr('liger')

# Convertir el DataFrame de pandas a un DataFrame de R
with localconverter(ro.default_converter + pandas2ri.converter):
    r_data = ro.conversion.py2rpy(df_data)

# Crear un objeto liger en R
ro.globalenv['r_data'] = r_data
ro.r('liger_object <- createLiger(list(batch1 = r_data))')

# Normalización y escalado de datos
ro.r('liger_object <- normalize(liger_object)')
ro.r('liger_object <- scaleNotCenter(liger_object)')

# Realizar la integración con liger
ro.r('liger_object <- optimizeALS(liger_object, k = 20)')
ro.r('liger_object <- quantileAlignSNF(liger_object)')

# Obtener los resultados de la integración
ro.r('integrated_matrix <- liger_object@H.norm')

# Convertir la matriz integrada de R a pandas
integrated_matrix = ro.conversion.rpy2py(ro.r('integrated_matrix'))

# Crear un nuevo objeto AnnData con la matriz integrada
adata_integrated = anndata.AnnData(X=integrated_matrix)

# Guardar el objeto AnnData integrado
adata_integrated.write('integrated_anndata.h5ad')

print("Integración con LIGER completada y guardada en 'integrated_anndata.h5ad'")



PackageNotInstalledError: The R package "liger" is not installed.